In [300]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import mlflow
import dvc

from sklearn.preprocessing import (
    OneHotEncoder,
    LabelEncoder,
    MinMaxScaler,
    StandardScaler
)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import (
    make_pipeline,
    Pipeline
)
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

In [256]:
import pandas as pd

uri = "https://raw.githubusercontent.com/SandeepSuthar169/Datasets/main/bank.csv"
df = pd.read_csv(uri)

print(df.head())



   age         job  marital  education default  balance housing loan  contact  \
0   59      admin.  married  secondary      no     2343     yes   no  unknown   
1   56      admin.  married  secondary      no       45      no   no  unknown   
2   41  technician  married  secondary      no     1270     yes   no  unknown   
3   55    services  married  secondary      no     2476     yes   no  unknown   
4   54      admin.  married   tertiary      no      184      no   no  unknown   

   day month  duration  campaign  pdays  previous poutcome deposit  
0    5   may      1042         1     -1         0  unknown     yes  
1    5   may      1467         1     -1         0  unknown     yes  
2    5   may      1389         1     -1         0  unknown     yes  
3    5   may       579         1     -1         0  unknown     yes  
4    5   may       673         2     -1         0  unknown     yes  


In [257]:
df.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
deposit      0
dtype: int64

In [258]:
df.duplicated().sum()

np.int64(0)

In [259]:
df.describe()

,age,balance,day,duration,campaign,pdays,previous
count,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000
mean,41.231948,1528.538524,15.658036,371.993818,2.508421,51.330407,0.832557
std,11.913369,3225.413326,8.420740,347.128386,2.722077,108.758282,2.292007
min,18.000000,-6847.000000,1.000000,2.000000,1.000000,-1.000000,0.000000
25%,32.000000,122.000000,8.000000,138.000000,1.000000,-1.000000,0.000000
50%,39.000000,550.000000,15.000000,255.000000,2.000000,-1.000000,0.000000
75%,49.000000,1708.000000,22.000000,496.000000,3.000000,20.750000,1.000000
max,95.000000,81204.000000,31.000000,3881.000000,63.000000,854.000000,58.000000


In [260]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.4+ MB


In [261]:
print(df['job'].value_counts())
print("\n")
print(df['marital'].value_counts())
print("\n")


job
management       2566
blue-collar      1944
technician       1823
admin.           1334
services          923
retired           778
self-employed     405
student           360
unemployed        357
entrepreneur      328
housemaid         274
unknown            70
Name: count, dtype: int64


marital
married     6351
single      3518
divorced    1293
Name: count, dtype: int64




In [262]:
print(df['education'].value_counts())
print("\n")
print(df['default'].value_counts())
print("\n")

education
secondary    5476
tertiary     3689
primary      1500
unknown       497
Name: count, dtype: int64


default
no     10994
yes      168
Name: count, dtype: int64




In [263]:
print(df['housing'].value_counts())
print("\n")
print(df['loan'].value_counts())
print("\n")

housing
no     5881
yes    5281
Name: count, dtype: int64


loan
no     9702
yes    1460
Name: count, dtype: int64




In [264]:
print(df['month'].value_counts())
print("\n")
print(df['poutcome'].value_counts())
print("\n")
print(df['deposit'].value_counts())


month
may    2824
aug    1519
jul    1514
jun    1222
nov     943
apr     923
feb     776
oct     392
jan     344
sep     319
mar     276
dec     110
Name: count, dtype: int64


poutcome
unknown    8326
failure    1228
success    1071
other       537
Name: count, dtype: int64


deposit
no     5873
yes    5289
Name: count, dtype: int64


In [265]:
'''
one-hot-encoding -> deposit, month, loan, housing, default
ordinal -encoding -> job, poutcome, education, marital

'''

'\none-hot-encoding -> deposit, month, loan, housing, default\nordinal -encoding -> job, poutcome, education, marital\n\n'

In [266]:
df['marital'] = df['marital'].replace({
    'divorced': 1,
    'single': 2,
    'married': 3
})
df['marital'] = df['marital'].astype(int)

C:\Users\Sande\AppData\Local\Temp\ipykernel_15784\1200014302.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['marital'] = df['marital'].replace({


In [268]:
df['job'] = df['job'].replace({
    'unknown': 1,
    'unemployed': 1,
    'housemaid': 2,
    'student': 2,
    'self-employed': 3,
    'retired': 4,
    'services': 4,
    'admin.': 5,
    'technician': 5,
    'blue-collar': 5,
    'management': 6,
    'entrepreneur': 6   
    })
df['job'] = df['job'].astype(int)

C:\Users\Sande\AppData\Local\Temp\ipykernel_15784\3550678970.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['job'] = df['job'].replace({


In [269]:
df['poutcome'] = df['poutcome'].replace({
    'unknown': 1,
    'other': 1,
    'failure': 2,
    'success': 2
})
df['poutcome'] = df['poutcome'].astype(int)

C:\Users\Sande\AppData\Local\Temp\ipykernel_15784\3120525272.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['poutcome'] = df['poutcome'].replace({


In [270]:
df['education'] = df['education'].replace({
    'unknown': 1,
    'primary': 2,
    'tertiary': 3,
    'secondary': 4
})
df['education'] = df['education'].astype(int)

C:\Users\Sande\AppData\Local\Temp\ipykernel_15784\3271902194.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['education'] = df['education'].replace({


In [271]:
for i, col in enumerate(df.columns):
    print(f"{i}, {col}")

0, age
1, job
2, marital
3, education
4, default
5, balance
6, housing
7, loan
8, contact
9, day
10, month
11, duration
12, campaign
13, pdays
14, previous
15, poutcome
16, deposit


In [272]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  int64 
 2   marital    11162 non-null  int64 
 3   education  11162 non-null  int64 
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  int64 
 16  deposit    11162 non-null  object
dtypes: int64(11), object(6)
memory usage: 1.4+ MB


In [273]:
df['contact'] = df['contact'].replace({
    'unknown': 1,
    'cellular': 2,
    'telephone': 3   
})
df['contact'] = df['contact'].astype(int)

C:\Users\Sande\AppData\Local\Temp\ipykernel_15784\1788602718.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['contact'] = df['contact'].replace({


In [292]:
#deposit, month, loan, housing, default
prepro= ColumnTransformer(transformers=[
    ('one', OneHotEncoder(), ['default', 'housing', 'loan', 'month'])
], 
     remainder='passthrough'
)

In [293]:
pipe = Pipeline(steps =[
    ('prepro', prepro),
    ("classi", RandomForestClassifier())
])

In [294]:
train_data,test_data=train_test_split(df, random_state=42, test_size=0.2)



In [297]:
X_train = train_data.drop(columns=['deposit'])
y_train = train_data['deposit']

X_test = test_data.drop(columns=['deposit'])
y_test = test_data['deposit']

In [298]:
pipe.fit(X_train, y_train)

c:\Users\Sande\Desktop\project\bank\.venv\Lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('prepro',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one', OneHotEncoder(),
                                                  ['default', 'housing', 'loan',
                                                   'month'])])),
                ('classi', RandomForestClassifier())])

In [299]:
y_pred = pipe.predict(X_test)


In [301]:
acc = accuracy_score(y_test,y_pred)
pre = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [303]:
print("acc:->",acc)
print('pre:->', pre)
print('recall:->', recall)
print('f1 :->', f1)

acc:-> 0.8468428123600538
pre:-> 0.8821892393320965
recall:-> 0.8156089193825042
f1 :-> 0.8475935828877005
